In [28]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "D:/ML/"
print(DOWNLOAD_ROOT)
DATASET_PATH = os.path.join("datasets", "Countries")
DIR_PATH = {"Afganistan":os.path.join(DATASET_PATH, "Afganistan"),
            "China":os.path.join(DATASET_PATH, "China"),
            "CAR":os.path.join(DATASET_PATH, "CAR"),
           "Ethiopia":os.path.join(DATASET_PATH, "Ethiopia"),
           "India":os.path.join(DATASET_PATH, "India"),
           "Iran":os.path.join(DATASET_PATH, "Iran"),
            "KNDR":os.path.join(DATASET_PATH, "KNDR"),
            "Libya":os.path.join(DATASET_PATH, "Libya"),
           "Morocco":os.path.join(DATASET_PATH, "Morocco"),
           "Sudan":os.path.join(DATASET_PATH, "Sudan"),
           "Syria":os.path.join(DATASET_PATH, "Syria"),
           "Turkey":os.path.join(DATASET_PATH, "Turkey"),
           }
filenames = {}
item_index_lab =[]
for items in DIR_PATH.keys():
    print(items)
    item_index_lab.append(items)
    filenames[items] = [name for name in sorted(os.listdir(DIR_PATH[items])) if len(name) > 20] 

D:/ML/
Afganistan
China
CAR
Ethiopia
India
Iran
KNDR
Libya
Morocco
Sudan
Syria
Turkey


In [29]:
from charset_normalizer import from_path
import string

def remove_chars_from_text(text, chars):
    # return "".join([ch for ch in text if ch not in chars])
    content = ''
    for ch in text:
        if ch not in chars:
            content = content + ''.join(ch)
        else:
            content = content + ''.join(' ')
    return content

import re

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)
spec_chars = string.punctuation + '\r' + '\n\xa0«»\t—…' 

x_temp = []
y_temp = []
for item in filenames.keys():
    post_temp=[]
    for names in filenames[item]:
        content = str(from_path(DIR_PATH[item]+"\\"+names).best()).lower()  
        content = remove_emojis(content)
        # content = re.sub(r'(\\u[0-9A-Fa-f]+)', lambda matchobj: chr(int(matchobj.group(0)[2:], 16)), content)
        content = remove_chars_from_text(content, spec_chars)
        content = remove_chars_from_text(content, string.digits)
        for i in range(1, 10):
            content = content.replace('  ', ' ')
        x_temp.append(content)
        y_temp.append(item)

In [30]:
import numpy as np
from sklearn.model_selection import train_test_split


X = np.array(x_temp, dtype=object)
y = np.array(y_temp)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
try:
    import urlextract # may require an Internet connection to download root domain names
    
    url_extractor = urlextract.URLExtract()
except ImportError:
    url_extractor = None

In [32]:
from sklearn.base import BaseEstimator, TransformerMixin
import urlextract # may require an Internet connection to download root domain names
import re
from collections import Counter
import nltk
from tokenizer_exceptions import normalizer_exc_rus
ru_stopwords = set(
        """
    а авось ага агу аж ай али алло ау ах ая
    б будем будет будете будешь буду будут будучи будь будьте бы был была были было
    быть бац без безусловно бишь благо благодаря ближайшие близко более больше
    будто бывает бывала бывали бываю бывают бытует
    в вам вами вас весь во вот все всё всего всей всем всём всеми всему всех всею
    всея всю вся вы ваш ваша ваше ваши вдали вдобавок вдруг ведь везде вернее
    взаимно взаправду видно вишь включая вместо внакладе вначале вне вниз внизу
    вновь вовсе возможно воистину вокруг вон вообще вопреки вперекор вплоть
    вполне вправду вправе впрочем впрямь вресноту вроде вряд всегда всюду
    всякий всякого всякой всячески вчеред
    г го где гораздо гав
    д да для до дабы давайте давно давным даже далее далеко дальше данная
    данного данное данной данном данному данные данный данных дану данунах
    даром де действительно довольно доколе доколь долго должен должна
    должно должны должный дополнительно другая другие другим другими
    других другое другой
    е его едим едят ее её ей ел ела ем ему емъ если ест есть ешь еще ещё ею едва
    ежели еле
    ж же
    з за затем зато зачем здесь значит зря
    и из или им ими имъ их ибо иль имеет имел имела имело именно иметь иначе
    иногда иным иными итак ишь
    й
    к как кем ко когда кого ком кому комья которая которого которое которой котором
    которому которою которую которые который которым которыми которых кто ка кабы
    каждая каждое каждые каждый кажется казалась казались казалось казался казаться
    какая какие каким какими каков какого какой какому какою касательно кой коли
    коль конечно короче кроме кстати ку куда
    л ли либо лишь любая любого любое любой любом любую любыми любых
    м меня мне мной мною мог моги могите могла могли могло могу могут мое моё моего
    моей моем моём моему моею можем может можете можешь мои мой моим моими моих
    мочь мою моя мы мало меж между менее меньше мимо многие много многого многое
    многом многому можно мол му
    н на нам нами нас наса наш наша наше нашего нашей нашем нашему нашею наши нашим
    нашими наших нашу не него нее неё ней нем нём нему нет нею ним ними них но
    наверняка наверху навряд навыворот над надо назад наиболее наизворот
    наизнанку наипаче накануне наконец наоборот наперед наперекор наподобие
    например напротив напрямую насилу настоящая настоящее настоящие настоящий
    насчет нате находиться начала начале неважно негде недавно недалеко незачем
    некем некогда некому некоторая некоторые некоторый некоторых некто некуда
    нельзя немногие немногим немного необходимо необходимости необходимые
    необходимым неоткуда непрерывно нередко несколько нету неужели нечего
    нечем нечему нечто нешто нибудь нигде ниже низко никак никакой никем
    никогда никого никому никто никуда ниоткуда нипочем ничего ничем ничему
    ничто ну нужная нужно нужного нужные нужный нужных ныне нынешнее нынешней
    нынешних нынче
    о об один одна одни одним одними одних одно одного одной одном одному одною
    одну он она оне они оно от оба общую обычно ого однажды однако ой около оный
    оп опять особенно особо особую особые откуда отнелижа отнелиже отовсюду
    отсюда оттого оттот оттуда отчего отчему ох очевидно очень ом
    п по при паче перед под подавно поди подобная подобно подобного подобные
    подобный подобным подобных поелику пожалуй пожалуйста позже поистине
    пока покамест поколе поколь покуда покудова помимо понеже поприще пор
    пора посему поскольку после посреди посредством потом потому потомушта
    похожем почему почти поэтому прежде притом причем про просто прочего
    прочее прочему прочими проще прям пусть
    р ради разве ранее рано раньше рядом
    с сам сама сами самим самими самих само самого самом самому саму свое своё
    своего своей своем своём своему своею свои свой своим своими своих свою своя
    себе себя собой собою самая самое самой самый самых сверх свыше се сего сей
    сейчас сие сих сквозь сколько скорее скоро следует слишком смогут сможет
    сначала снова со собственно совсем сперва спокону спустя сразу среди сродни
    стал стала стали стало стать суть сызнова
    та то ту ты ти так такая такие таким такими таких такого такое такой таком такому такою
    такую те тебе тебя тем теми тех тобой тобою того той только том томах тому
    тот тою также таки таков такова там твои твоим твоих твой твоя твоё
    теперь тогда тоже тотчас точно туда тут тьфу тая
    у уже увы уж ура ух ую
    ф фу
    х ха хе хорошо хотел хотела хотелось хотеть хоть хотя хочешь хочу хуже
    ч чего чем чём чему что чтобы часто чаще чей через чтоб чуть чхать чьим
    чьих чьё чё
    ш ша
    щ ща щас
    ы ых ые ый
    э эта эти этим этими этих это этого этой этом этому этот этою эту эдак эдакий
    эй эка экий этак этакий эх
    ю
    я явно явных яко якобы якоже
    
    и что не это  быть этот это свой как - этот весь быть что ▫ но г. %
    """.split()
    )
# stemmer = nltk.PorterStemmer()
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")
def Make_dict(X_text, strip_headers=True, lower_case=True,
    remove_punctuation=True, replace_urls=True, replace_numbers=True,
    stemming=True, remove_stopwords=True):
    
    X_transformed = []
    for email in X_text:
        text = email or ""
        text = normalizer_exc_rus(text)
        if lower_case:
            text = text.lower()
        if replace_urls and url_extractor is not None:
            urls = list(set(url_extractor.find_urls(text)))
            urls.sort(key=lambda url: len(url), reverse=True)
            for url in urls:
                text = text.replace(url, "")
        if replace_numbers:
            text = re.sub(r'\d+(?:\.\d*)?(?:[eE][+-]?\d+)?', '', text)
        if remove_punctuation:
            text = re.sub(r'\W+', ' ', text, flags=re.M)
        if remove_stopwords:
            filtered_words = []
            for token in text.split():
                if token not in ru_stopwords:
                    filtered_words.append(token)
    
            # Join the filtered words to form a clean text
            text = ' '.join(filtered_words)
        word_counts = Counter(text.split())
        if stemming and stemmer is not None:
            stemmed_word_counts = Counter()
            for word, count in word_counts.items():
                stemmed_word = stemmer.stem(word)
                stemmed_word_counts[stemmed_word] += count
            word_counts = stemmed_word_counts
        X_transformed.append(word_counts)
    return np.array(X_transformed)

In [33]:
X_train_dic = Make_dict(X_train, stemming=False)
X_train_words = []
for post in X_train_dic:
    for word in post.keys():
        X_train_words.append(word)
print(X_train_words [:7])


['распп', 'приглашает', 'предпринимателей', 'сферы', 'креативных', 'индустрий', 'принять']


In [34]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(vocabulary = set(X_train_words))
X_train_transformed = count.fit_transform(X_train)

In [35]:
X_train_transformed.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log_clf = LogisticRegression(solver="lbfgs", max_iter=5000, random_state=42)
score = cross_val_score(log_clf, X_train_transformed, y_train, cv=3, verbose=3)
score.mean()

[CV] END ................................ score: (test=0.890) total time=   1.0s
[CV] END ................................ score: (test=0.897) total time=   1.2s
[CV] END ................................ score: (test=0.888) total time=   1.3s


0.8915379300494971

In [37]:
count.get_feature_names_out()

array(['a', 'aaa', 'aamiddleeast', ..., '፲፪ቱ', '፳፬', 'ℹ'], dtype=object)

In [38]:
from sklearn.metrics import precision_score, recall_score

X_test_transformed = count.transform(X_test)

log_clf = LogisticRegression(solver="lbfgs", max_iter=6000, random_state=42)
log_clf.fit(X_test_transformed, y_test)

y_pred = log_clf.predict(X_test_transformed)

print("Precision: {:.2f}%".format(100 * precision_score(y_test, y_pred, average='micro')))
print("Recall: {:.2f}%".format(100 * recall_score(y_test, y_pred, average='micro')))

Precision: 99.43%
Recall: 99.43%


In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

count_ngram = CountVectorizer(ngram_range=(1,3))
X_train_transformed = count_ngram.fit_transform(X_train)


log_clf_ngram = LogisticRegression(solver="lbfgs", max_iter=5000, random_state=42)
score = cross_val_score(log_clf_ngram, X_train_transformed, y_train, cv=3, verbose=3)
score.mean()

[CV] END ................................ score: (test=0.862) total time=  26.1s
[CV] END ................................ score: (test=0.849) total time=  27.3s
[CV] END ................................ score: (test=0.858) total time=  30.8s


0.856256069547599

In [40]:
# count_ngram.vocabulary_.keys()

In [41]:
from sklearn.metrics import precision_score, recall_score
count_ngram_test = CountVectorizer(ngram_range=(1,3), vocabulary = count_ngram.vocabulary_.keys() )
X_test_transformed_ngram = count_ngram_test.transform(X_test)

log_clf_ngram.fit(X_test_transformed_ngram, y_test)

y_pred_ngram = log_clf_ngram.predict(X_test_transformed_ngram)

print("Precision: {:.2f}%".format(100 * precision_score(y_test, y_pred_ngram, average='micro')))
print("Recall: {:.2f}%".format(100 * recall_score(y_test, y_pred_ngram, average='micro')))

Precision: 99.43%
Recall: 99.43%


In [48]:
new_text = ['Нигер последовал примеру Мали, разорвав связи с Украиной после того, как пресс-секретарь разведывательного агентства Министерства обороны Украины признал, что Украина оказывала поддержку повстанцам, которые убили малийских вооруженных сил и российских агентов 25-27 июля в Тинзауатене, на севере Мали, недалеко от границы с Алжиром.Посол Украины в Сенегале также выразил безоговорочную поддержку малийским повстанцам, что привело к вызову посла в МИД Сенегала.',
            'Еще новости афганской промышленности. В Кандагаре спустя 18 лет возобновила работу текстильная фабрика. Официальные лица говорят, что они отремонтировали оборудование на фабрике за шесть месяцев, сообщают афганское СМИ. По словам работников фабрики, с возобновлением работы фабрики созданы рабочие места. 69-летний Мохаммад, проработавший на текстильной фабрике в Кандагаре значительное время, рад вернуться на работу.',
           'Накопленное непонимание между Китаем и США не может быть решено лишь одним подобным изящным жестом. Страны расходятся по широкому кругу вопросов, санкционный механизм против Китая не ослаблен. Поэтому отправка панд – это скорее демонстрация миролюбивой позиции Китая в противовес США.',
            'В Алжире, как правило, на похоронах мужчины сидят около дома, хорошо, если есть сад или терраса, а женщины в доме, надеть платок женщина должна обязательно, даже если в обычной жизни она не покрывает голову. Если покойный жил в квартире, то прямо во дворе ставят стулья и шатры со столами для мужчин.Готовят либо нанятые кухарки, либо родственницы покойного. Пока составляла пост от подруги услышала,что в их семье часто еду приносят те, кто приходит в дом, а готовить должны невестки, а не дочери умершего. Мой муж сказал, что это совершенно необязательно, на похоронах его бабушки готовили только её дочери, это было их желание и никаких особых правил на счёт этого нет. Обязательного блюда на поминки в Алжире нет. Народ приходит помянутьот трех дней до недели, если у человека была большая семья и много знакомых!Кормят традиционными блюдами: суп-шорба, кус-кус/беркукес/тлитли с мясом и т.д.Могилу посещают каждое утро, в течении трех дней. Через 40 дней устанавливают небольшое надгробие, никаких памятников,вычурных элементов. На надгробии имя, фамилия, даты рождения и смерти, фотографий нет. Вроде бы написала всё, что знала. Задавайте вопросы в комментариях, если таковые имеются.',
            'Сегодня в Венесуэле проходят президентские выборы. Мой коллега Дмитрий Морозов рассуждает, как их итоги могут повлиять на дальнейший путь страны. Вот самое важное из его статьи.📍В выборах один тур, участвуют десять кандидатов, однако только двое имеют реальные шансы на победу: действующий президент Николас Мадуро и Эдмундо Гонсалес Уррутия, кандидат от Единой демократической платформы, объединяющей наиболее значимые оппозиционные партии.'
           ]
x_temp = []
for post_temp in new_text:
        content = str(post_temp).lower()  
        content = remove_emojis(content)
        content = remove_chars_from_text(content, spec_chars)
        content = remove_chars_from_text(content, string.digits)
        for i in range(1, 10):
            content = content.replace('  ', ' ')
        x_temp.append(content)
   

count = CountVectorizer(vocabulary = set(X_train_words))
X_t = count.transform(x_temp)
accuracy = log_clf.predict(X_t)
print (accuracy)
yhat = log_clf.predict_proba(X_t)
# summarize the predicted probabilities
for item_index in range(0, len(x_temp)):
    print (x_temp[item_index])
    for item in range(0, len(item_index_lab)):
        print('Predicted Probabilities: {} - {}'.format(log_clf.classes_[item], yhat[item_index][item]))

['CAR' 'Afganistan' 'China' 'India' 'China']
нигер последовал примеру мали разорвав связи с украиной после того как пресс секретарь разведывательного агентства министерства обороны украины признал что украина оказывала поддержку повстанцам которые убили малийских вооруженных сил и российских агентов июля в тинзауатене на севере мали недалеко от границы с алжиром посол украины в сенегале также выразил безоговорочную поддержку малийским повстанцам что привело к вызову посла в мид сенегала 
Predicted Probabilities: Afganistan - 0.04404923758109427
Predicted Probabilities: CAR - 0.3662073893246939
Predicted Probabilities: China - 0.11412805915068364
Predicted Probabilities: Ethiopia - 0.045636686382374024
Predicted Probabilities: India - 0.10983586234945501
Predicted Probabilities: Iran - 0.08317589292190163
Predicted Probabilities: KNDR - 0.02956894156686255
Predicted Probabilities: Libya - 0.02434875905850931
Predicted Probabilities: Morocco - 0.040838366272583145
Predicted Probabilities

In [49]:
new_text = ['Нигер последовал примеру Мали, разорвав связи с Украиной после того, как пресс-секретарь разведывательного агентства Министерства обороны Украины признал, что Украина оказывала поддержку повстанцам, которые убили малийских вооруженных сил и российских агентов 25-27 июля в Тинзауатене, на севере Мали, недалеко от границы с Алжиром.Посол Украины в Сенегале также выразил безоговорочную поддержку малийским повстанцам, что привело к вызову посла в МИД Сенегала.',
            'Еще новости афганской промышленности. В Кандагаре спустя 18 лет возобновила работу текстильная фабрика. Официальные лица говорят, что они отремонтировали оборудование на фабрике за шесть месяцев, сообщают афганское СМИ. По словам работников фабрики, с возобновлением работы фабрики созданы рабочие места. 69-летний Мохаммад, проработавший на текстильной фабрике в Кандагаре значительное время, рад вернуться на работу.',
           'Накопленное непонимание между Китаем и США не может быть решено лишь одним подобным изящным жестом. Страны расходятся по широкому кругу вопросов, санкционный механизм против Китая не ослаблен. Поэтому отправка панд – это скорее демонстрация миролюбивой позиции Китая в противовес США.',
            'В Алжире, как правило, на похоронах мужчины сидят около дома, хорошо, если есть сад или терраса, а женщины в доме, надеть платок женщина должна обязательно, даже если в обычной жизни она не покрывает голову. Если покойный жил в квартире, то прямо во дворе ставят стулья и шатры со столами для мужчин.Готовят либо нанятые кухарки, либо родственницы покойного. Пока составляла пост от подруги услышала,что в их семье часто еду приносят те, кто приходит в дом, а готовить должны невестки, а не дочери умершего. Мой муж сказал, что это совершенно необязательно, на похоронах его бабушки готовили только её дочери, это было их желание и никаких особых правил на счёт этого нет. Обязательного блюда на поминки в Алжире нет. Народ приходит помянутьот трех дней до недели, если у человека была большая семья и много знакомых!Кормят традиционными блюдами: суп-шорба, кус-кус/беркукес/тлитли с мясом и т.д.Могилу посещают каждое утро, в течении трех дней. Через 40 дней устанавливают небольшое надгробие, никаких памятников,вычурных элементов. На надгробии имя, фамилия, даты рождения и смерти, фотографий нет. Вроде бы написала всё, что знала. Задавайте вопросы в комментариях, если таковые имеются.',
            'Сегодня в Венесуэле проходят президентские выборы. Мой коллега Дмитрий Морозов рассуждает, как их итоги могут повлиять на дальнейший путь страны. Вот самое важное из его статьи.📍В выборах один тур, участвуют десять кандидатов, однако только двое имеют реальные шансы на победу: действующий президент Николас Мадуро и Эдмундо Гонсалес Уррутия, кандидат от Единой демократической платформы, объединяющей наиболее значимые оппозиционные партии.'
           ]
x_temp = []
for post_temp in new_text:
        content = str(post_temp).lower()  
        content = remove_emojis(content)
        content = remove_chars_from_text(content, spec_chars)
        content = remove_chars_from_text(content, string.digits)
        for i in range(1, 10):
            content = content.replace('  ', ' ')
        x_temp.append(content)
   
count = CountVectorizer(vocabulary = set(X_train_words))
X_t = count_ngram_test.transform(x_temp)

accuracy_ngram = log_clf_ngram.predict(X_t)
print (accuracy_ngram)
yhat = log_clf_ngram.predict_proba(X_t)
# summarize the predicted probabilities
for item_index in range(0, len(x_temp)):
    print (x_temp[item_index])
    for item in range(0, len(item_index_lab)):
        print('Predicted Probabilities: {} - {}'.format(log_clf_ngram.classes_[item], yhat[item_index][item]))

['CAR' 'Afganistan' 'China' 'India' 'China']
нигер последовал примеру мали разорвав связи с украиной после того как пресс секретарь разведывательного агентства министерства обороны украины признал что украина оказывала поддержку повстанцам которые убили малийских вооруженных сил и российских агентов июля в тинзауатене на севере мали недалеко от границы с алжиром посол украины в сенегале также выразил безоговорочную поддержку малийским повстанцам что привело к вызову посла в мид сенегала 
Predicted Probabilities: Afganistan - 0.06606036370499471
Predicted Probabilities: CAR - 0.25672254221535384
Predicted Probabilities: China - 0.24831860078220605
Predicted Probabilities: Ethiopia - 0.03956728055492642
Predicted Probabilities: India - 0.06682174787195705
Predicted Probabilities: Iran - 0.08789940847503791
Predicted Probabilities: KNDR - 0.018178517738515827
Predicted Probabilities: Libya - 0.023026586542236407
Predicted Probabilities: Morocco - 0.018397831483485914
Predicted Probabiliti

In [18]:
DOWNLOAD_ROOT = "D:/ML/"
print(DOWNLOAD_ROOT)
DATASET_PATH_test = os.path.join("datasets", "spam")
DIR_PATH_test = {"Africa":os.path.join(DATASET_PATH_test, "Africa"),
            "China":os.path.join(DATASET_PATH_test, "China"),
            "MiddleEAST":os.path.join(DATASET_PATH_test, "MiddleEAST"),
           "LatAmerica":os.path.join(DATASET_PATH_test, "LatAmerica"),
           "IranPakistanAfganistan":os.path.join(DATASET_PATH_test, "IranPakistanAfganistan"),}
filenames = {}
for items in DIR_PATH.keys():
    print(items)
    filenames[items] = [name for name in sorted(os.listdir(DIR_PATH[items])) if len(name) > 20] 

D:/ML/
Afganistan
China
CAR
Ethiopia
India
Iran
KNDR
Libya
Morocco
Sudan
Syria
Turkey


In [19]:
from charset_normalizer import from_path
import string

def remove_chars_from_text(text, chars):
    # return "".join([ch for ch in text if ch not in chars])
    content = ''
    for ch in text:
        if ch not in chars:
            content = content + ''.join(ch)
        else:
            content = content + ''.join(' ')
    return content

import re

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)
spec_chars = string.punctuation + '\r' + '\n\xa0«»\t—…' 

x_test = []
for item in filenames.keys():
    post_temp=[]
    for names in filenames[item]:
        content = str(from_path(DIR_PATH[item]+"\\"+names).best()).lower()  
        content = remove_emojis(content)
        # content = re.sub(r'(\\u[0-9A-Fa-f]+)', lambda matchobj: chr(int(matchobj.group(0)[2:], 16)), content)
        content = remove_chars_from_text(content, spec_chars)
        content = remove_chars_from_text(content, string.digits)
        for i in range(1, 10):
            content = content.replace('  ', ' ')
        x_test.append(content)       

In [51]:
x_temp = []
for post_temp in x_test[-20:]:
        content = str(post_temp).lower()  
        content = remove_emojis(content)
        content = remove_chars_from_text(content, spec_chars)
        content = remove_chars_from_text(content, string.digits)
        for i in range(1, 10):
            content = content.replace('  ', ' ')
        x_temp.append(content)
   
count = CountVectorizer(vocabulary = set(X_train_words))
X_t = count.transform(x_temp)

accuracy = log_clf.predict(X_t)
print (accuracy)
yhat = log_clf.predict_proba(X_t)
# summarize the predicted probabilities
for item_index in range(0, len(x_temp)):
    print (x_temp[item_index])
    for item in range(0, len(item_index_lab)):
        print('Predicted Probabilities: {} - {}'.format(log_clf.classes_[item], yhat[item_index][item]))

['India' 'Turkey' 'Turkey' 'Turkey' 'Turkey' 'China' 'Turkey' 'Turkey'
 'Turkey' 'Turkey' 'China' 'Turkey' 'Turkey' 'India' 'Turkey' 'Turkey'
 'Turkey' 'Turkey' 'China' 'China']
тема девочки нарин не утихает обсуждается и обсуждается а всё потому что все подозреваемые ведут себя крайне профессионально сейчас единственный признавший вину невзат бахтияр сосед который прятал тело за предложенные тыс турецких лир а все остальные говорят что не виноваты и все совпадения случайны или ошибочны и еще я заметил что вину начинают перекидывать на невзата даже отец вчера вечером в интервью говорил что не верит что его брат мог это сделать а бахтияр мой учитывая что он все дней искал девочку при том что сам спрятал тело и приходил к отцу и желал ему терпения мог потому что так лицемерить может только ужасный человек еще есть одна из новых версий что девочка не увидела что то недопустимое а была случайно убита братом которого родители решили спасти чтобы и его не потерять ибо его посадят жизнь его б

In [57]:
import pandas as pd
x_temp = []
for post_temp in x_test[-200:-150]:
        content = str(post_temp).lower()  
        content = remove_emojis(content)
        content = remove_chars_from_text(content, spec_chars)
        content = remove_chars_from_text(content, string.digits)
        for i in range(1, 10):
            content = content.replace('  ', ' ')
        x_temp.append(content)
   

count = CountVectorizer(vocabulary = set(X_train_words))
X_t = count_ngram_test.transform(x_temp)

accuracy_ngram = log_clf_ngram.predict(X_t)
print (accuracy_ngram)

df = pd.DataFrame(log_clf_ngram.predict_proba(X_t), columns=log_clf_ngram.classes_)
df_max = pd.DataFrame({'predict': log_clf_ngram.predict(X_t), 'accuracy': df.max(axis=1)})
df_max

['Turkey' 'India' 'Turkey' 'Turkey' 'Turkey' 'Turkey' 'Turkey' 'China'
 'Turkey' 'China' 'China' 'Turkey' 'Turkey' 'Turkey' 'Turkey' 'India'
 'Turkey' 'Turkey' 'Turkey' 'Turkey' 'Turkey' 'CAR' 'Turkey' 'Turkey'
 'Syria' 'India' 'Turkey' 'Turkey' 'Turkey' 'Turkey' 'India' 'China'
 'Turkey' 'India' 'China' 'Turkey' 'China' 'Turkey' 'Turkey' 'Turkey'
 'Turkey' 'Turkey' 'China' 'India' 'India' 'India' 'Turkey' 'Turkey'
 'China' 'India']


,predict,accuracy
0,Turkey,0.416426
1,India,0.228391
2,Turkey,0.951885
3,Turkey,0.913303
4,Turkey,0.346618
5,Turkey,0.999729
6,Turkey,0.994293
7,China,0.528232
8,Turkey,0.936126
9,China,0.328324
